## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import censusdata as cs
from census import Census
import warnings

pd.set_option('display.max_rows',None, 'display.max_columns',None,'display.max_colwidth', None)

### Pass API Key

In [2]:
c = Census("e0577a26a616f4dda60446eae987e3b6d0d944a3")

### Preview Variable

In [3]:
cs.printtable(cs.censustable('acs5',2019,'B19013'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B19013_001E  | MEDIAN HOUSEHOLD INCOME IN THE | !! Estimate Median household income in the past 12 month | int  
-------------------------------------------------------------------------------------------------------------------


### Set Geographies

- Arlington
- Tarrant County
- Dallas Fort Worth
- Texas

In [4]:
geographies = {
    'Arlington, TX': {'for': 'place:04000', 'in': 'state:48'},  
    'Tarrant County, TX': {'for': 'county:439', 'in': 'state:48'},
    'Dallas-Fort Worth-Arlington MSA': {'for': 'metropolitan statistical area/micropolitan statistical area:19100'},
    'Texas State': {'for': 'state:48'}
}

### Get Data

- **ACS 5-Year Estimate**

In [5]:
AllData = []

Years = [2010,2020]

for year in Years:

    for geo in geographies:

        juris = geographies[geo]
        ACS = c.acs5.get(['B19013_001E'],juris,year=year)

        Data = pd.DataFrame(ACS)
        Data['Geography'] = geo
        Data['Year'] = year
        

        AllData.append(Data)

combinedData = pd.concat(AllData,ignore_index=True)

combinedData = combinedData[['Year', 'Geography', 'B19013_001E']]
combinedData.rename(columns={'B19013_001E': 'Median Household Income'}, inplace=True)

DataPivot = combinedData.pivot(index='Geography', columns='Year', values='Median Household Income')


# Grab Arlington Median and divide by Median for other geographies 
DataPivot['Arlington Compared 2010'] = (DataPivot.loc['Arlington, TX', 2010] / DataPivot[2010]) * 100 
DataPivot['Arlington Compared 2020'] = (DataPivot.loc['Arlington, TX', 2020] / DataPivot[2020]) * 100

# Set Arlington's number to NA
DataPivot.loc['Arlington, TX', 'Arlington Compared 2010'] = np.nan
DataPivot.loc['Arlington, TX', 'Arlington Compared 2020'] = np.nan

# Absolute difference (change in income)
DataPivot['Absolute Change Income (2020-2010)'] = DataPivot[2020] - DataPivot[2010]

# Percentage change
DataPivot['Percentage Change Income (2010-2020)'] = ((DataPivot[2020] / DataPivot[2010]) - 1) * 100


# Format $
DataPivotFormat = DataPivot.copy()
DataPivotFormat[2010] = DataPivotFormat[2010].apply(lambda x: "${:,.0f}".format(x))   
DataPivotFormat[2020] = DataPivotFormat[2020].apply(lambda x: "${:,.0f}".format(x))
DataPivotFormat['Absolute Change Income (2020-2010)'] = DataPivotFormat['Absolute Change Income (2020-2010)'].apply(lambda x: "${:,.0f}".format(x))


# Format %
DataPivotFormat['Arlington Compared 2010'] = DataPivotFormat['Arlington Compared 2010'].apply(
lambda x: "{:.0f}%".format(x) if pd.notnull(x) else "")

DataPivotFormat['Arlington Compared 2020'] = DataPivotFormat['Arlington Compared 2020'].apply(
    lambda x: "{:.0f}%".format(x) if pd.notnull(x) else "")

DataPivotFormat['Percentage Change Income (2010-2020)'] = DataPivotFormat['Percentage Change Income (2010-2020)'].apply(
    lambda x: "{:.1f}%".format(x) if pd.notnull(x) else "")



DataPivotFormat.rename(index={'Arlington, TX': 'City of Arlington, TX'}, inplace=True)

DataPivotFormat.reset_index(inplace= True)
DataPivotFormat.columns.name = None 

DataPivotFormat['Geography'] = DataPivotFormat['Geography'].replace({'Arlington, TX': 'City of Arlington'})

DataPivotFormat.to_csv('Median Household Income.csv', index = False)

In [6]:
DataPivotFormat

,Geography,2010,2020,Arlington Compared 2010,Arlington Compared 2020,Absolute Change Income (2020-2010),Percentage Change Income (2010-2020)
0,"City of Arlington, TX","$52,094","$63,351",,,"$11,257",21.6%
1,Dallas-Fort Worth-Arlington MSA,"$56,349","$72,882",92%,87%,"$16,533",29.3%
2,"Tarrant County, TX","$55,306","$70,306",94%,90%,"$15,000",27.1%
3,Texas State,"$49,646","$63,826",105%,99%,"$14,180",28.6%
